In [1]:
#Establish required libraries

import sys
import types
import pandas as pd 
from botocore.client import Config
import ibm_boto3


import numpy as np
import pandas as pd
from pandas import Series, DataFrame, Index
from IPython.display import Image

def __iter__(self): return 0

import pandas as pd
import numpy as np 

class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)


In [2]:
# The code was removed by Watson Studio for sharing.

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 385636 entries, 0 to 385635
Data columns (total 72 columns):
Demo_Key                                    385636 non-null int64
Promo_ID                                    385636 non-null object
Promo_CPG_Name                              281228 non-null object
Promo_Name                                  385636 non-null object
Retailer_Promo_Contract_ID                  0 non-null float64
Retailer_Name                               385636 non-null object
Retailer_Store_ID                           385636 non-null object
Product_Unit_UPC                            385636 non-null object
Product_Group_ID                            385635 non-null object
Product_Brand                               385636 non-null object
Product_Brand_Category_Desc                 385636 non-null object
Product_Category                            385636 non-null object
Product_Sub_Category                        385636 non-null object
Product_Type           

In [3]:
#select the fields you need
df_price_elasticity = df_data_1[['Promo_ID','Promo_Name','Retailer_Name','Retailer_Store_ID','Product_Category',
                                 'Product_Sub_Category',
                             'Product_Unit_UPC','Product_Group_ID','Product_Brand','Product_Brand_Category_Desc',
                              'Product_Unit_Description','Retailer_Product_Base_Day_Unit_Volume',
                              'TPP_Unit_Sale_Price','Retailer_Product_Regular_Unit_Price','TPP_Incremental_Cases',
                             'TPP_Baseline_Cases_Volume','TPP_Type','TPP_Start','TPP_End','Product_Units_Per_Case','Product_CPG_COGS_Price']]






In [4]:
#Visually inspect the different products in the table
df_price_elasticity.Product_Sub_Category.value_counts(dropna = False).sort_index()

Cereal         67360
Cleaners       30312
Ice Cream      47152
Trail Mix      74096
Vegetables    116196
Yogurt         50520
Name: Product_Sub_Category, dtype: int64

In [5]:
df_price_elasticity.shape

(385636, 21)

In [6]:
#De-Dup the file
df_price_elasticity=df_price_elasticity.drop_duplicates(subset=['Promo_ID','Retailer_Store_ID','Product_Unit_UPC'], keep='first', inplace=False)
df_price_elasticity.shape




(385636, 21)

In [7]:
#remove all products but the one we us

df_price_elasticity=df_price_elasticity[df_price_elasticity['Product_Sub_Category'] =='Ice Cream']
df_price_elasticity.shape

(47152, 21)

In [8]:
#create a new field for size
df_price_elasticity['SIZE']=pd.DataFrame(df_price_elasticity['Product_Group_ID'].str[-2:])


In [9]:
df_price_elasticity.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47152 entries, 234076 to 281227
Data columns (total 22 columns):
Promo_ID                                 47152 non-null object
Promo_Name                               47152 non-null object
Retailer_Name                            47152 non-null object
Retailer_Store_ID                        47152 non-null object
Product_Category                         47152 non-null object
Product_Sub_Category                     47152 non-null object
Product_Unit_UPC                         47152 non-null object
Product_Group_ID                         47151 non-null object
Product_Brand                            47152 non-null object
Product_Brand_Category_Desc              47152 non-null object
Product_Unit_Description                 47152 non-null object
Retailer_Product_Base_Day_Unit_Volume    47152 non-null float64
TPP_Unit_Sale_Price                      47152 non-null float64
Retailer_Product_Regular_Unit_Price      47152 non-null float64


In [10]:
#create a copy of the original data set
sale=df_price_elasticity

In [11]:
sale.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47152 entries, 234076 to 281227
Data columns (total 22 columns):
Promo_ID                                 47152 non-null object
Promo_Name                               47152 non-null object
Retailer_Name                            47152 non-null object
Retailer_Store_ID                        47152 non-null object
Product_Category                         47152 non-null object
Product_Sub_Category                     47152 non-null object
Product_Unit_UPC                         47152 non-null object
Product_Group_ID                         47151 non-null object
Product_Brand                            47152 non-null object
Product_Brand_Category_Desc              47152 non-null object
Product_Unit_Description                 47152 non-null object
Retailer_Product_Base_Day_Unit_Volume    47152 non-null float64
TPP_Unit_Sale_Price                      47152 non-null float64
Retailer_Product_Regular_Unit_Price      47152 non-null float64


In [12]:
# create rename the sale price to price, create the cases and units field, drop fields you don't need anymore
#label all of the records as sale records

sale=sale.rename(index=str, columns={"TPP_Unit_Sale_Price": "PRICE"})
sale['CASES'] = sale['TPP_Baseline_Cases_Volume'] + sale['TPP_Incremental_Cases'] 
sale['GROUP']='ONSALE'
sale['UNITS']=sale['CASES']*sale['Product_Units_Per_Case']
sale=sale.drop(['TPP_Baseline_Cases_Volume', 'TPP_Incremental_Cases','Retailer_Product_Regular_Unit_Price','Retailer_Product_Base_Day_Unit_Volume'], axis=1)

In [13]:
sale.info()

<class 'pandas.core.frame.DataFrame'>
Index: 47152 entries, 234076 to 281227
Data columns (total 21 columns):
Promo_ID                       47152 non-null object
Promo_Name                     47152 non-null object
Retailer_Name                  47152 non-null object
Retailer_Store_ID              47152 non-null object
Product_Category               47152 non-null object
Product_Sub_Category           47152 non-null object
Product_Unit_UPC               47152 non-null object
Product_Group_ID               47151 non-null object
Product_Brand                  47152 non-null object
Product_Brand_Category_Desc    47152 non-null object
Product_Unit_Description       47152 non-null object
PRICE                          47152 non-null float64
TPP_Type                       47152 non-null object
TPP_Start                      47152 non-null object
TPP_End                        47152 non-null object
Product_Units_Per_Case         47152 non-null float64
Product_CPG_COGS_Price         47152 non

In [14]:
#copy the orginal data set, create the non-sale prices
nosale=df_price_elasticity


In [15]:
nosale.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47152 entries, 234076 to 281227
Data columns (total 22 columns):
Promo_ID                                 47152 non-null object
Promo_Name                               47152 non-null object
Retailer_Name                            47152 non-null object
Retailer_Store_ID                        47152 non-null object
Product_Category                         47152 non-null object
Product_Sub_Category                     47152 non-null object
Product_Unit_UPC                         47152 non-null object
Product_Group_ID                         47151 non-null object
Product_Brand                            47152 non-null object
Product_Brand_Category_Desc              47152 non-null object
Product_Unit_Description                 47152 non-null object
Retailer_Product_Base_Day_Unit_Volume    47152 non-null float64
TPP_Unit_Sale_Price                      47152 non-null float64
Retailer_Product_Regular_Unit_Price      47152 non-null float64


In [16]:
#create the cases and price field
nosale=nosale.rename(index=str, columns={"TPP_Baseline_Cases_Volume": "CASES", "Retailer_Product_Regular_Unit_Price":"PRICE"})


In [17]:
nosale.info()

<class 'pandas.core.frame.DataFrame'>
Index: 47152 entries, 234076 to 281227
Data columns (total 22 columns):
Promo_ID                                 47152 non-null object
Promo_Name                               47152 non-null object
Retailer_Name                            47152 non-null object
Retailer_Store_ID                        47152 non-null object
Product_Category                         47152 non-null object
Product_Sub_Category                     47152 non-null object
Product_Unit_UPC                         47152 non-null object
Product_Group_ID                         47151 non-null object
Product_Brand                            47152 non-null object
Product_Brand_Category_Desc              47152 non-null object
Product_Unit_Description                 47152 non-null object
Retailer_Product_Base_Day_Unit_Volume    47152 non-null float64
TPP_Unit_Sale_Price                      47152 non-null float64
PRICE                                    47152 non-null float64
TPP_I

In [18]:
#Calculate the number of Units
nosale['UNITS']=nosale['CASES']*nosale['Product_Units_Per_Case']

In [19]:
#identify the prices as non-sale prices.
nosale['GROUP']='NOTONSALE'


#Remove stuff you don't need


nosale=nosale.drop([ 'TPP_Incremental_Cases','TPP_Unit_Sale_Price','Retailer_Product_Base_Day_Unit_Volume','TPP_Type'], axis=1)
nosale['TPP_Type']='N'

In [20]:
nosale.head()

,Promo_ID,Promo_Name,Retailer_Name,Retailer_Store_ID,Product_Category,Product_Sub_Category,Product_Unit_UPC,Product_Group_ID,Product_Brand,Product_Brand_Category_Desc,...,PRICE,CASES,TPP_Start,TPP_End,Product_Units_Per_Case,Product_CPG_COGS_Price,SIZE,UNITS,GROUP,TPP_Type
234076,Brandega-48335,Days of Summer,Botann's,12345-94249-107,Frozen,Ice Cream,77220-10116,FFF-F-IC-P-64,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 16 oz,...,2.41,3.0,2019-07-07,2019-07-13,12.0,1.68,64,36.0,NOTONSALE,N
234077,Brandega-48335,Days of Summer,Botann's,12345-94249-107,Frozen,Ice Cream,77220-10164,FFF-F-IC-P-16,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 64 oz,...,6.83,5.0,2019-07-07,2019-07-13,6.0,4.75,16,30.0,NOTONSALE,N
234078,Brandega-48335,Days of Summer,Botann's,12345-94249-107,Frozen,Ice Cream,77220-10216,FFF-F-IC-P-64,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 16 oz,...,2.41,3.0,2019-07-07,2019-07-13,12.0,1.68,64,36.0,NOTONSALE,N
234079,Brandega-48335,Days of Summer,Botann's,12345-94249-107,Frozen,Ice Cream,77220-10264,FFF-F-IC-P-16,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 64 oz,...,6.83,5.0,2019-07-07,2019-07-13,6.0,4.75,16,30.0,NOTONSALE,N
234080,Brandega-48335,Days of Summer,Botann's,12345-94249-107,Frozen,Ice Cream,77220-10316,FFF-F-IC-P-64,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 16 oz,...,2.41,3.0,2019-07-07,2019-07-13,12.0,1.68,64,36.0,NOTONSALE,N


In [21]:
sale.info()
nosale.info()

<class 'pandas.core.frame.DataFrame'>
Index: 47152 entries, 234076 to 281227
Data columns (total 21 columns):
Promo_ID                       47152 non-null object
Promo_Name                     47152 non-null object
Retailer_Name                  47152 non-null object
Retailer_Store_ID              47152 non-null object
Product_Category               47152 non-null object
Product_Sub_Category           47152 non-null object
Product_Unit_UPC               47152 non-null object
Product_Group_ID               47151 non-null object
Product_Brand                  47152 non-null object
Product_Brand_Category_Desc    47152 non-null object
Product_Unit_Description       47152 non-null object
PRICE                          47152 non-null float64
TPP_Type                       47152 non-null object
TPP_Start                      47152 non-null object
TPP_End                        47152 non-null object
Product_Units_Per_Case         47152 non-null float64
Product_CPG_COGS_Price         47152 non

In [22]:
#include needed fields
nosale = nosale[['Promo_ID',
 'Promo_Name',
 'Retailer_Name',
 'Retailer_Store_ID',
 'Product_Group_ID',
 'Product_Unit_UPC',
 'Product_Brand',
 'Product_Brand_Category_Desc',
 'Product_Unit_Description',
 'PRICE',
 'TPP_Type',
 'TPP_Start',
 'TPP_End',
 'Product_Units_Per_Case',
 'Product_CPG_COGS_Price',
 'SIZE',
 'CASES',
 'GROUP',
 'UNITS']]

In [23]:
#include needed fields
sale = sale[['Promo_ID',
 'Promo_Name',
 'Retailer_Name',
 'Retailer_Store_ID',
 'Product_Group_ID',
 'Product_Unit_UPC',
 'Product_Brand',
 'Product_Brand_Category_Desc',
 'Product_Unit_Description',
 'PRICE',
 'TPP_Type',
 'TPP_Start',
 'TPP_End',
 'Product_Units_Per_Case',
 'Product_CPG_COGS_Price',
 'SIZE',
 'CASES',
 'GROUP',
 'UNITS']]

In [24]:
sale.info()
nosale.info()

<class 'pandas.core.frame.DataFrame'>
Index: 47152 entries, 234076 to 281227
Data columns (total 19 columns):
Promo_ID                       47152 non-null object
Promo_Name                     47152 non-null object
Retailer_Name                  47152 non-null object
Retailer_Store_ID              47152 non-null object
Product_Group_ID               47151 non-null object
Product_Unit_UPC               47152 non-null object
Product_Brand                  47152 non-null object
Product_Brand_Category_Desc    47152 non-null object
Product_Unit_Description       47152 non-null object
PRICE                          47152 non-null float64
TPP_Type                       47152 non-null object
TPP_Start                      47152 non-null object
TPP_End                        47152 non-null object
Product_Units_Per_Case         47152 non-null float64
Product_CPG_COGS_Price         47152 non-null float64
SIZE                           47151 non-null object
CASES                          47152 no

In [25]:
#append the sale and non-sale records into one dataframe
df_total=pd.DataFrame(sale.append(nosale, ignore_index=True, verify_integrity=False))

In [26]:
#convert the tpp_start which is character to a pandas date field
df_total['date'] = pd.to_datetime(df_total['TPP_Start'])
df_total.head()

,Promo_ID,Promo_Name,Retailer_Name,Retailer_Store_ID,Product_Group_ID,Product_Unit_UPC,Product_Brand,Product_Brand_Category_Desc,Product_Unit_Description,PRICE,TPP_Type,TPP_Start,TPP_End,Product_Units_Per_Case,Product_CPG_COGS_Price,SIZE,CASES,GROUP,UNITS,date
0,Brandega-48335,Days of Summer,Botann's,12345-94249-107,FFF-F-IC-P-64,77220-10116,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 16 oz,Frank's Famous Flavors Black Walnut 16 oz,2.13,F,2019-07-07,2019-07-13,12.0,1.68,64,6.0,ONSALE,72.0,2019-07-07
1,Brandega-48335,Days of Summer,Botann's,12345-94249-107,FFF-F-IC-P-16,77220-10164,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 64 oz,Frank's Famous Flavors Black Walnut 64 oz,6.04,F,2019-07-07,2019-07-13,6.0,4.75,16,12.0,ONSALE,72.0,2019-07-07
2,Brandega-48335,Days of Summer,Botann's,12345-94249-107,FFF-F-IC-P-64,77220-10216,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 16 oz,Frank's Famous Flavors Burgundy Cherry 16 oz,2.13,F,2019-07-07,2019-07-13,12.0,1.68,64,6.0,ONSALE,72.0,2019-07-07
3,Brandega-48335,Days of Summer,Botann's,12345-94249-107,FFF-F-IC-P-16,77220-10264,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 64 oz,Frank's Famous Flavors Burgundy Cherry 64 oz,6.04,F,2019-07-07,2019-07-13,6.0,4.75,16,12.0,ONSALE,72.0,2019-07-07
4,Brandega-48335,Days of Summer,Botann's,12345-94249-107,FFF-F-IC-P-64,77220-10316,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 16 oz,Frank's Famous Flavors Butterscotch 16 oz,2.13,F,2019-07-07,2019-07-13,12.0,1.68,64,6.0,ONSALE,72.0,2019-07-07


In [27]:
#create a field for year and month
df_total['year'], df_total['month'] = df_total['date'].dt.year, df_total['date'].dt.month
df_total.head()

,Promo_ID,Promo_Name,Retailer_Name,Retailer_Store_ID,Product_Group_ID,Product_Unit_UPC,Product_Brand,Product_Brand_Category_Desc,Product_Unit_Description,PRICE,...,TPP_End,Product_Units_Per_Case,Product_CPG_COGS_Price,SIZE,CASES,GROUP,UNITS,date,year,month
0,Brandega-48335,Days of Summer,Botann's,12345-94249-107,FFF-F-IC-P-64,77220-10116,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 16 oz,Frank's Famous Flavors Black Walnut 16 oz,2.13,...,2019-07-13,12.0,1.68,64,6.0,ONSALE,72.0,2019-07-07,2019,7
1,Brandega-48335,Days of Summer,Botann's,12345-94249-107,FFF-F-IC-P-16,77220-10164,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 64 oz,Frank's Famous Flavors Black Walnut 64 oz,6.04,...,2019-07-13,6.0,4.75,16,12.0,ONSALE,72.0,2019-07-07,2019,7
2,Brandega-48335,Days of Summer,Botann's,12345-94249-107,FFF-F-IC-P-64,77220-10216,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 16 oz,Frank's Famous Flavors Burgundy Cherry 16 oz,2.13,...,2019-07-13,12.0,1.68,64,6.0,ONSALE,72.0,2019-07-07,2019,7
3,Brandega-48335,Days of Summer,Botann's,12345-94249-107,FFF-F-IC-P-16,77220-10264,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 64 oz,Frank's Famous Flavors Burgundy Cherry 64 oz,6.04,...,2019-07-13,6.0,4.75,16,12.0,ONSALE,72.0,2019-07-07,2019,7
4,Brandega-48335,Days of Summer,Botann's,12345-94249-107,FFF-F-IC-P-64,77220-10316,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 16 oz,Frank's Famous Flavors Butterscotch 16 oz,2.13,...,2019-07-13,12.0,1.68,64,6.0,ONSALE,72.0,2019-07-07,2019,7


In [28]:
#assign random numbers to each retailer
wookie = pd.DataFrame(df_total.groupby(['Retailer_Store_ID'])['PRICE'].mean())
wookie=wookie.drop(['PRICE'], axis=1)


In [29]:
wookie['Retailer_R'] = (np.random.randint(-30, 60, wookie.shape[0]))/1000
wookie.reset_index(level=0, inplace=True)
wookie.head()


,Retailer_Store_ID,Retailer_R
0,12345-10024-126,0.014
1,12345-14213-136,0.026
2,12345-14601-157,0.030
3,12345-15225-129,-0.015
4,12345-15225-167,0.013


In [30]:
df_total1 =df_total.merge(wookie, on=['Retailer_Store_ID'], how='inner')
df_total1.head()

,Promo_ID,Promo_Name,Retailer_Name,Retailer_Store_ID,Product_Group_ID,Product_Unit_UPC,Product_Brand,Product_Brand_Category_Desc,Product_Unit_Description,PRICE,...,Product_Units_Per_Case,Product_CPG_COGS_Price,SIZE,CASES,GROUP,UNITS,date,year,month,Retailer_R
0,Brandega-48335,Days of Summer,Botann's,12345-94249-107,FFF-F-IC-P-64,77220-10116,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 16 oz,Frank's Famous Flavors Black Walnut 16 oz,2.13,...,12.0,1.68,64,6.0,ONSALE,72.0,2019-07-07,2019,7,-0.012
1,Brandega-48335,Days of Summer,Botann's,12345-94249-107,FFF-F-IC-P-16,77220-10164,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 64 oz,Frank's Famous Flavors Black Walnut 64 oz,6.04,...,6.0,4.75,16,12.0,ONSALE,72.0,2019-07-07,2019,7,-0.012
2,Brandega-48335,Days of Summer,Botann's,12345-94249-107,FFF-F-IC-P-64,77220-10216,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 16 oz,Frank's Famous Flavors Burgundy Cherry 16 oz,2.13,...,12.0,1.68,64,6.0,ONSALE,72.0,2019-07-07,2019,7,-0.012
3,Brandega-48335,Days of Summer,Botann's,12345-94249-107,FFF-F-IC-P-16,77220-10264,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 64 oz,Frank's Famous Flavors Burgundy Cherry 64 oz,6.04,...,6.0,4.75,16,12.0,ONSALE,72.0,2019-07-07,2019,7,-0.012
4,Brandega-48335,Days of Summer,Botann's,12345-94249-107,FFF-F-IC-P-64,77220-10316,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 16 oz,Frank's Famous Flavors Butterscotch 16 oz,2.13,...,12.0,1.68,64,6.0,ONSALE,72.0,2019-07-07,2019,7,-0.012


In [31]:
#assign random numbers to each product group id
df_total1.Product_Group_ID.value_counts(dropna = False).sort_index()

FFF-F-IC-P-16    47150
FFF-F-IC-P-64    47152
NaN                  2
Name: Product_Group_ID, dtype: int64

In [32]:
 wookie = pd.DataFrame(df_total1.groupby(['Product_Group_ID'])['PRICE'].mean())
wookie=wookie.drop(['PRICE'], axis=1)

In [33]:
wookie['PRODUCT_GROUP_ID_R'] = (np.random.randint(-30, 60, wookie.shape[0]))/1000
wookie.reset_index(level=0, inplace=True)
wookie.head()

,Product_Group_ID,PRODUCT_GROUP_ID_R
0,FFF-F-IC-P-16,0.007
1,FFF-F-IC-P-64,-0.012


In [34]:
#!pip install plotly --upgrade

In [35]:
df_total2 =df_total1.merge(wookie, on=['Product_Group_ID'], how='inner')
df_total2.head()

,Promo_ID,Promo_Name,Retailer_Name,Retailer_Store_ID,Product_Group_ID,Product_Unit_UPC,Product_Brand,Product_Brand_Category_Desc,Product_Unit_Description,PRICE,...,Product_CPG_COGS_Price,SIZE,CASES,GROUP,UNITS,date,year,month,Retailer_R,PRODUCT_GROUP_ID_R
0,Brandega-48335,Days of Summer,Botann's,12345-94249-107,FFF-F-IC-P-64,77220-10116,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 16 oz,Frank's Famous Flavors Black Walnut 16 oz,2.13,...,1.68,64,6.0,ONSALE,72.0,2019-07-07,2019,7,-0.012,-0.012
1,Brandega-48335,Days of Summer,Botann's,12345-94249-107,FFF-F-IC-P-64,77220-10216,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 16 oz,Frank's Famous Flavors Burgundy Cherry 16 oz,2.13,...,1.68,64,6.0,ONSALE,72.0,2019-07-07,2019,7,-0.012,-0.012
2,Brandega-48335,Days of Summer,Botann's,12345-94249-107,FFF-F-IC-P-64,77220-10316,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 16 oz,Frank's Famous Flavors Butterscotch 16 oz,2.13,...,1.68,64,6.0,ONSALE,72.0,2019-07-07,2019,7,-0.012,-0.012
3,Brandega-48335,Days of Summer,Botann's,12345-94249-107,FFF-F-IC-P-64,77220-10416,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 16 oz,Frank's Famous Flavors Chocolate Chip 16 oz,2.13,...,1.68,64,6.0,ONSALE,72.0,2019-07-07,2019,7,-0.012,-0.012
4,Brandega-48335,Days of Summer,Botann's,12345-94249-107,FFF-F-IC-P-64,77220-10516,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 16 oz,Frank's Famous Flavors Chocolate Mint 16 oz,2.13,...,1.68,64,6.0,ONSALE,72.0,2019-07-07,2019,7,-0.012,-0.012


In [36]:
#assign random numbers to each product unit upc value
df_total2.Product_Unit_UPC.value_counts(dropna = False).sort_index()

77220-10116    3368
77220-10164    3368
77220-10216    3368
77220-10264    3368
77220-10316    3368
77220-10364    3368
77220-10416    3368
77220-10464    3368
77220-10516    3368
77220-10564    3368
77220-10616    3368
77220-10664    3368
77220-10716    3368
77220-10764    3368
77220-10816    3368
77220-10864    3368
77220-10916    3368
77220-10964    3368
77220-11016    3368
77220-11064    3368
77220-11116    3368
77220-11164    3368
77220-11216    3368
77220-11264    3368
77220-11316    3368
77220-11364    3368
77220-11416    3368
77220-11464    3366
Name: Product_Unit_UPC, dtype: int64

In [37]:
wookie = pd.DataFrame(df_total2.groupby(['Product_Unit_UPC'])['PRICE'].mean())
wookie=wookie.drop(['PRICE'], axis=1)

wookie['Product_Unit_UPC_R'] = (np.random.randint(-30, 60, wookie.shape[0]))/1000
wookie.reset_index(level=0, inplace=True)
wookie.head()
df_total3 =df_total2.merge(wookie, on=['Product_Unit_UPC'], how='inner')
df_total3.head()

,Promo_ID,Promo_Name,Retailer_Name,Retailer_Store_ID,Product_Group_ID,Product_Unit_UPC,Product_Brand,Product_Brand_Category_Desc,Product_Unit_Description,PRICE,...,SIZE,CASES,GROUP,UNITS,date,year,month,Retailer_R,PRODUCT_GROUP_ID_R,Product_Unit_UPC_R
0,Brandega-48335,Days of Summer,Botann's,12345-94249-107,FFF-F-IC-P-64,77220-10116,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 16 oz,Frank's Famous Flavors Black Walnut 16 oz,2.13,...,64,6.0,ONSALE,72.0,2019-07-07,2019,7,-0.012,-0.012,0.05
1,Brandega-52846,Quench Your Hunger,Botann's,12345-94249-107,FFF-F-IC-P-64,77220-10116,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 16 oz,Frank's Famous Flavors Black Walnut 16 oz,2.22,...,64,4.0,ONSALE,48.0,2019-09-26,2019,9,-0.012,-0.012,0.05
2,Brandega-55965,Be Mine Valentine,Botann's,12345-94249-107,FFF-F-IC-P-64,77220-10116,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 16 oz,Frank's Famous Flavors Black Walnut 16 oz,2.10,...,64,11.0,ONSALE,132.0,2019-02-05,2019,2,-0.012,-0.012,0.05
3,Brandega-77062,Dial Back Savings,Botann's,12345-94249-107,FFF-F-IC-P-64,77220-10116,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 16 oz,Frank's Famous Flavors Black Walnut 16 oz,2.22,...,64,19.0,ONSALE,228.0,2019-04-23,2019,4,-0.012,-0.012,0.05
4,Brandega-48335,Days of Summer,Botann's,12345-94249-107,FFF-F-IC-P-64,77220-10116,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 16 oz,Frank's Famous Flavors Black Walnut 16 oz,2.41,...,64,3.0,NOTONSALE,36.0,2019-07-07,2019,7,-0.012,-0.012,0.05


In [38]:
#assign random numbers to each promo
df_total2.Promo_Name.value_counts(dropna = False).sort_index()

Be Mine Valentine     23576
Days of Summer        23576
Dial Back Savings     23574
Quench Your Hunger    23576
Name: Promo_Name, dtype: int64

In [39]:
wookie = pd.DataFrame(df_total3.groupby(['Promo_Name'])['PRICE'].mean())
wookie=wookie.drop(['PRICE'], axis=1)

wookie['Promo_Name_R'] = (np.random.randint(-30, 60, wookie.shape[0]))/1000
wookie.reset_index(level=0, inplace=True)
wookie.head()
df_total4 =df_total3.merge(wookie, on=['Promo_Name'], how='inner')
df_total4.head()

,Promo_ID,Promo_Name,Retailer_Name,Retailer_Store_ID,Product_Group_ID,Product_Unit_UPC,Product_Brand,Product_Brand_Category_Desc,Product_Unit_Description,PRICE,...,CASES,GROUP,UNITS,date,year,month,Retailer_R,PRODUCT_GROUP_ID_R,Product_Unit_UPC_R,Promo_Name_R
0,Brandega-48335,Days of Summer,Botann's,12345-94249-107,FFF-F-IC-P-64,77220-10116,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 16 oz,Frank's Famous Flavors Black Walnut 16 oz,2.13,...,6.0,ONSALE,72.0,2019-07-07,2019,7,-0.012,-0.012,0.05,-0.023
1,Brandega-48335,Days of Summer,Botann's,12345-94249-107,FFF-F-IC-P-64,77220-10116,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 16 oz,Frank's Famous Flavors Black Walnut 16 oz,2.41,...,3.0,NOTONSALE,36.0,2019-07-07,2019,7,-0.012,-0.012,0.05,-0.023
2,Brandega-48335,Days of Summer,Botann's,12345-79914-108,FFF-F-IC-P-64,77220-10116,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 16 oz,Frank's Famous Flavors Black Walnut 16 oz,2.13,...,6.0,ONSALE,72.0,2019-07-07,2019,7,0.011,-0.012,0.05,-0.023
3,Brandega-48335,Days of Summer,Botann's,12345-79914-108,FFF-F-IC-P-64,77220-10116,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 16 oz,Frank's Famous Flavors Black Walnut 16 oz,2.41,...,3.0,NOTONSALE,36.0,2019-07-07,2019,7,0.011,-0.012,0.05,-0.023
4,Brandega-48335,Days of Summer,Botann's,12345-90005-109,FFF-F-IC-P-64,77220-10116,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 16 oz,Frank's Famous Flavors Black Walnut 16 oz,2.13,...,6.0,ONSALE,72.0,2019-07-07,2019,7,-0.027,-0.012,0.05,-0.023


In [40]:
#assign different values to sale and non-sale records
df_total4['GROUP_R'] = np.where(df_total4['GROUP'] == 'ONSALE', -0.10, 0.01)
df_total4.head()

,Promo_ID,Promo_Name,Retailer_Name,Retailer_Store_ID,Product_Group_ID,Product_Unit_UPC,Product_Brand,Product_Brand_Category_Desc,Product_Unit_Description,PRICE,...,GROUP,UNITS,date,year,month,Retailer_R,PRODUCT_GROUP_ID_R,Product_Unit_UPC_R,Promo_Name_R,GROUP_R
0,Brandega-48335,Days of Summer,Botann's,12345-94249-107,FFF-F-IC-P-64,77220-10116,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 16 oz,Frank's Famous Flavors Black Walnut 16 oz,2.13,...,ONSALE,72.0,2019-07-07,2019,7,-0.012,-0.012,0.05,-0.023,-0.10
1,Brandega-48335,Days of Summer,Botann's,12345-94249-107,FFF-F-IC-P-64,77220-10116,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 16 oz,Frank's Famous Flavors Black Walnut 16 oz,2.41,...,NOTONSALE,36.0,2019-07-07,2019,7,-0.012,-0.012,0.05,-0.023,0.01
2,Brandega-48335,Days of Summer,Botann's,12345-79914-108,FFF-F-IC-P-64,77220-10116,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 16 oz,Frank's Famous Flavors Black Walnut 16 oz,2.13,...,ONSALE,72.0,2019-07-07,2019,7,0.011,-0.012,0.05,-0.023,-0.10
3,Brandega-48335,Days of Summer,Botann's,12345-79914-108,FFF-F-IC-P-64,77220-10116,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 16 oz,Frank's Famous Flavors Black Walnut 16 oz,2.41,...,NOTONSALE,36.0,2019-07-07,2019,7,0.011,-0.012,0.05,-0.023,0.01
4,Brandega-48335,Days of Summer,Botann's,12345-90005-109,FFF-F-IC-P-64,77220-10116,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 16 oz,Frank's Famous Flavors Black Walnut 16 oz,2.13,...,ONSALE,72.0,2019-07-07,2019,7,-0.027,-0.012,0.05,-0.023,-0.10


In [41]:
df_total4.month.value_counts(dropna = False).sort_index()

2    23576
4    23574
7    23576
9    23576
Name: month, dtype: int64

In [42]:
#assign values to each month
conditions = [
    (df_total4['month'] == 2),
    (df_total4['month'] ==4),
    (df_total4['month'] ==7)]
choices = [-0.03, 0.02, 0.06]
df_total4['month_R'] = np.select(conditions, choices, default=0.02)


In [43]:
df_total4.head()

,Promo_ID,Promo_Name,Retailer_Name,Retailer_Store_ID,Product_Group_ID,Product_Unit_UPC,Product_Brand,Product_Brand_Category_Desc,Product_Unit_Description,PRICE,...,UNITS,date,year,month,Retailer_R,PRODUCT_GROUP_ID_R,Product_Unit_UPC_R,Promo_Name_R,GROUP_R,month_R
0,Brandega-48335,Days of Summer,Botann's,12345-94249-107,FFF-F-IC-P-64,77220-10116,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 16 oz,Frank's Famous Flavors Black Walnut 16 oz,2.13,...,72.0,2019-07-07,2019,7,-0.012,-0.012,0.05,-0.023,-0.10,0.06
1,Brandega-48335,Days of Summer,Botann's,12345-94249-107,FFF-F-IC-P-64,77220-10116,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 16 oz,Frank's Famous Flavors Black Walnut 16 oz,2.41,...,36.0,2019-07-07,2019,7,-0.012,-0.012,0.05,-0.023,0.01,0.06
2,Brandega-48335,Days of Summer,Botann's,12345-79914-108,FFF-F-IC-P-64,77220-10116,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 16 oz,Frank's Famous Flavors Black Walnut 16 oz,2.13,...,72.0,2019-07-07,2019,7,0.011,-0.012,0.05,-0.023,-0.10,0.06
3,Brandega-48335,Days of Summer,Botann's,12345-79914-108,FFF-F-IC-P-64,77220-10116,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 16 oz,Frank's Famous Flavors Black Walnut 16 oz,2.41,...,36.0,2019-07-07,2019,7,0.011,-0.012,0.05,-0.023,0.01,0.06
4,Brandega-48335,Days of Summer,Botann's,12345-90005-109,FFF-F-IC-P-64,77220-10116,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 16 oz,Frank's Famous Flavors Black Walnut 16 oz,2.13,...,72.0,2019-07-07,2019,7,-0.027,-0.012,0.05,-0.023,-0.10,0.06


In [44]:
#generate a random number for each row that is either 1 or -1
import random
df_total4['randNumCol'] = np.random.choice([1, -1], df_total4.shape[0])
df_total4.head(100)

,Promo_ID,Promo_Name,Retailer_Name,Retailer_Store_ID,Product_Group_ID,Product_Unit_UPC,Product_Brand,Product_Brand_Category_Desc,Product_Unit_Description,PRICE,...,date,year,month,Retailer_R,PRODUCT_GROUP_ID_R,Product_Unit_UPC_R,Promo_Name_R,GROUP_R,month_R,randNumCol
0,Brandega-48335,Days of Summer,Botann's,12345-94249-107,FFF-F-IC-P-64,77220-10116,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 16 oz,Frank's Famous Flavors Black Walnut 16 oz,2.13,...,2019-07-07,2019,7,-0.012,-0.012,0.05,-0.023,-0.10,0.06,-1
1,Brandega-48335,Days of Summer,Botann's,12345-94249-107,FFF-F-IC-P-64,77220-10116,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 16 oz,Frank's Famous Flavors Black Walnut 16 oz,2.41,...,2019-07-07,2019,7,-0.012,-0.012,0.05,-0.023,0.01,0.06,1
2,Brandega-48335,Days of Summer,Botann's,12345-79914-108,FFF-F-IC-P-64,77220-10116,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 16 oz,Frank's Famous Flavors Black Walnut 16 oz,2.13,...,2019-07-07,2019,7,0.011,-0.012,0.05,-0.023,-0.10,0.06,1
3,Brandega-48335,Days of Summer,Botann's,12345-79914-108,FFF-F-IC-P-64,77220-10116,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 16 oz,Frank's Famous Flavors Black Walnut 16 oz,2.41,...,2019-07-07,2019,7,0.011,-0.012,0.05,-0.023,0.01,0.06,-1
4,Brandega-48335,Days of Summer,Botann's,12345-90005-109,FFF-F-IC-P-64,77220-10116,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 16 oz,Frank's Famous Flavors Black Walnut 16 oz,2.13,...,2019-07-07,2019,7,-0.027,-0.012,0.05,-0.023,-0.10,0.06,1
5,Brandega-48335,Days of Summer,Botann's,12345-90005-109,FFF-F-IC-P-64,77220-10116,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 16 oz,Frank's Famous Flavors Black Walnut 16 oz,2.41,...,2019-07-07,2019,7,-0.027,-0.012,0.05,-0.023,0.01,0.06,-1
6,Brandega-48335,Days of Summer,Botann's,12345-33233-110,FFF-F-IC-P-64,77220-10116,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 16 oz,Frank's Famous Flavors Black Walnut 16 oz,2.13,...,2019-07-07,2019,7,-0.004,-0.012,0.05,-0.023,-0.10,0.06,1
7,Brandega-48335,Days of Summer,Botann's,12345-33233-110,FFF-F-IC-P-64,77220-10116,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 16 oz,Frank's Famous Flavors Black Walnut 16 oz,2.41,...,2019-07-07,2019,7,-0.004,-0.012,0.05,-0.023,0.01,0.06,-1
8,Brandega-48335,Days of Summer,Botann's,12345-28816-111,FFF-F-IC-P-64,77220-10116,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 16 oz,Frank's Famous Flavors Black Walnut 16 oz,2.13,...,2019-07-07,2019,7,-0.008,-0.012,0.05,-0.023,-0.10,0.06,-1
9,Brandega-48335,Days of Summer,Botann's,12345-28816-111,FFF-F-IC-P-64,77220-10116,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 16 oz,Frank's Famous Flavors Black Walnut 16 oz,2.41,...,2019-07-07,2019,7,-0.008,-0.012,0.05,-0.023,0.01,0.06,1


In [45]:
#sum the generated numbers for each row, making a unique value for each row called total_r

cols = ['Retailer_R','PRODUCT_GROUP_ID_R','Product_Unit_UPC_R','Promo_Name_R','GROUP_R','month_R']
df_total4['TOTAL_R'] = df_total4[cols].astype(float).sum(axis=1)


In [46]:
df_total4.head()

,Promo_ID,Promo_Name,Retailer_Name,Retailer_Store_ID,Product_Group_ID,Product_Unit_UPC,Product_Brand,Product_Brand_Category_Desc,Product_Unit_Description,PRICE,...,year,month,Retailer_R,PRODUCT_GROUP_ID_R,Product_Unit_UPC_R,Promo_Name_R,GROUP_R,month_R,randNumCol,TOTAL_R
0,Brandega-48335,Days of Summer,Botann's,12345-94249-107,FFF-F-IC-P-64,77220-10116,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 16 oz,Frank's Famous Flavors Black Walnut 16 oz,2.13,...,2019,7,-0.012,-0.012,0.05,-0.023,-0.10,0.06,-1,-0.037
1,Brandega-48335,Days of Summer,Botann's,12345-94249-107,FFF-F-IC-P-64,77220-10116,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 16 oz,Frank's Famous Flavors Black Walnut 16 oz,2.41,...,2019,7,-0.012,-0.012,0.05,-0.023,0.01,0.06,1,0.073
2,Brandega-48335,Days of Summer,Botann's,12345-79914-108,FFF-F-IC-P-64,77220-10116,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 16 oz,Frank's Famous Flavors Black Walnut 16 oz,2.13,...,2019,7,0.011,-0.012,0.05,-0.023,-0.10,0.06,1,-0.014
3,Brandega-48335,Days of Summer,Botann's,12345-79914-108,FFF-F-IC-P-64,77220-10116,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 16 oz,Frank's Famous Flavors Black Walnut 16 oz,2.41,...,2019,7,0.011,-0.012,0.05,-0.023,0.01,0.06,-1,0.096
4,Brandega-48335,Days of Summer,Botann's,12345-90005-109,FFF-F-IC-P-64,77220-10116,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 16 oz,Frank's Famous Flavors Black Walnut 16 oz,2.13,...,2019,7,-0.027,-0.012,0.05,-0.023,-0.10,0.06,1,-0.052


In [47]:
df_total4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 94302 entries, 0 to 94301
Data columns (total 30 columns):
Promo_ID                       94302 non-null object
Promo_Name                     94302 non-null object
Retailer_Name                  94302 non-null object
Retailer_Store_ID              94302 non-null object
Product_Group_ID               94302 non-null object
Product_Unit_UPC               94302 non-null object
Product_Brand                  94302 non-null object
Product_Brand_Category_Desc    94302 non-null object
Product_Unit_Description       94302 non-null object
PRICE                          94302 non-null float64
TPP_Type                       94302 non-null object
TPP_Start                      94302 non-null object
TPP_End                        94302 non-null object
Product_Units_Per_Case         94302 non-null float64
Product_CPG_COGS_Price         94302 non-null float64
SIZE                           94302 non-null object
CASES                          94302 non

In [48]:
#create new prices based on the adjustment numbers generated on the previous lines
df_total4['PRICE_R']=df_total4['PRICE']*(1.0-(df_total4['randNumCol']*df_total4['TOTAL_R']))
df_total4['UNITS_R']=df_total4['UNITS']*(1.0+(df_total4['randNumCol']*df_total4['TOTAL_R']))

In [49]:
df_total4.head()

,Promo_ID,Promo_Name,Retailer_Name,Retailer_Store_ID,Product_Group_ID,Product_Unit_UPC,Product_Brand,Product_Brand_Category_Desc,Product_Unit_Description,PRICE,...,Retailer_R,PRODUCT_GROUP_ID_R,Product_Unit_UPC_R,Promo_Name_R,GROUP_R,month_R,randNumCol,TOTAL_R,PRICE_R,UNITS_R
0,Brandega-48335,Days of Summer,Botann's,12345-94249-107,FFF-F-IC-P-64,77220-10116,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 16 oz,Frank's Famous Flavors Black Walnut 16 oz,2.13,...,-0.012,-0.012,0.05,-0.023,-0.10,0.06,-1,-0.037,2.05119,74.664
1,Brandega-48335,Days of Summer,Botann's,12345-94249-107,FFF-F-IC-P-64,77220-10116,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 16 oz,Frank's Famous Flavors Black Walnut 16 oz,2.41,...,-0.012,-0.012,0.05,-0.023,0.01,0.06,1,0.073,2.23407,38.628
2,Brandega-48335,Days of Summer,Botann's,12345-79914-108,FFF-F-IC-P-64,77220-10116,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 16 oz,Frank's Famous Flavors Black Walnut 16 oz,2.13,...,0.011,-0.012,0.05,-0.023,-0.10,0.06,1,-0.014,2.15982,70.992
3,Brandega-48335,Days of Summer,Botann's,12345-79914-108,FFF-F-IC-P-64,77220-10116,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 16 oz,Frank's Famous Flavors Black Walnut 16 oz,2.41,...,0.011,-0.012,0.05,-0.023,0.01,0.06,-1,0.096,2.64136,32.544
4,Brandega-48335,Days of Summer,Botann's,12345-90005-109,FFF-F-IC-P-64,77220-10116,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 16 oz,Frank's Famous Flavors Black Walnut 16 oz,2.13,...,-0.027,-0.012,0.05,-0.023,-0.10,0.06,1,-0.052,2.24076,68.256


In [50]:
#Make sure that size is always positive, create a new variable called price_size convert size to a numeric value
df_total4.SIZE = df_total4.SIZE.astype(float).fillna(0.0)


df_total4['PRICE_SIZE']=df_total4['PRICE_R']*df_total4['SIZE']


df_total4.SIZE.value_counts(dropna = False).sort_index()

16.0    47150
64.0    47152
Name: SIZE, dtype: int64

In [51]:
#create a dummy variable for ice cream
df_total4['SIZE_16'] = np.where(df_total4['SIZE'] == 16, 1, 0)
df_total4['SIZE_16_PRICE']=df_total4['SIZE_16']*df_total4['PRICE']
df_total4.head()

,Promo_ID,Promo_Name,Retailer_Name,Retailer_Store_ID,Product_Group_ID,Product_Unit_UPC,Product_Brand,Product_Brand_Category_Desc,Product_Unit_Description,PRICE,...,Promo_Name_R,GROUP_R,month_R,randNumCol,TOTAL_R,PRICE_R,UNITS_R,PRICE_SIZE,SIZE_16,SIZE_16_PRICE
0,Brandega-48335,Days of Summer,Botann's,12345-94249-107,FFF-F-IC-P-64,77220-10116,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 16 oz,Frank's Famous Flavors Black Walnut 16 oz,2.13,...,-0.023,-0.10,0.06,-1,-0.037,2.05119,74.664,131.27616,0,0.0
1,Brandega-48335,Days of Summer,Botann's,12345-94249-107,FFF-F-IC-P-64,77220-10116,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 16 oz,Frank's Famous Flavors Black Walnut 16 oz,2.41,...,-0.023,0.01,0.06,1,0.073,2.23407,38.628,142.98048,0,0.0
2,Brandega-48335,Days of Summer,Botann's,12345-79914-108,FFF-F-IC-P-64,77220-10116,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 16 oz,Frank's Famous Flavors Black Walnut 16 oz,2.13,...,-0.023,-0.10,0.06,1,-0.014,2.15982,70.992,138.22848,0,0.0
3,Brandega-48335,Days of Summer,Botann's,12345-79914-108,FFF-F-IC-P-64,77220-10116,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 16 oz,Frank's Famous Flavors Black Walnut 16 oz,2.41,...,-0.023,0.01,0.06,-1,0.096,2.64136,32.544,169.04704,0,0.0
4,Brandega-48335,Days of Summer,Botann's,12345-90005-109,FFF-F-IC-P-64,77220-10116,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 16 oz,Frank's Famous Flavors Black Walnut 16 oz,2.13,...,-0.023,-0.10,0.06,1,-0.052,2.24076,68.256,143.40864,0,0.0


In [52]:
#bring in the stats library to build the model
import statsmodels.api as sm  
#scikit learn is another library that you could use.  Its focus is more on prediction.  This library is more focused on understanding why and how certain variables impact your target.  Which is quantity in this case


/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [53]:
df_total4=df_total4.drop(['PRICE'], axis=1)

In [54]:
v = df_total4[['PRICE_R','TOTAL_R','SIZE','PRICE_SIZE','UNITS_R']]
w = df_total4[['PRICE_R','TOTAL_R','SIZE','PRICE_SIZE','UNITS_R']]
x = df_total4[['PRICE_R','TOTAL_R','SIZE','PRICE_SIZE','UNITS_R']]
y = df_total4[['PRICE_R','TOTAL_R','SIZE','PRICE_SIZE','UNITS_R']]
z = df_total4[['PRICE_R','TOTAL_R','SIZE','PRICE_SIZE','UNITS_R']]

In [55]:
pricev = pd.DataFrame((v['PRICE_R']*0.95))
pricew = pd.DataFrame((w['PRICE_R']*0.85))
pricey = pd.DataFrame((y['PRICE_R']*0.90))
pricez= pd.DataFrame((z['PRICE_R']*0.80))

qv = pd.DataFrame((v['UNITS_R']*1.05))
qw = pd.DataFrame((w['UNITS_R']*1.15))
qy = pd.DataFrame((y['UNITS_R']*1.10))
qz= pd.DataFrame((z['UNITS_R']*1.20))

In [56]:
df_total4.head()

,Promo_ID,Promo_Name,Retailer_Name,Retailer_Store_ID,Product_Group_ID,Product_Unit_UPC,Product_Brand,Product_Brand_Category_Desc,Product_Unit_Description,TPP_Type,...,Promo_Name_R,GROUP_R,month_R,randNumCol,TOTAL_R,PRICE_R,UNITS_R,PRICE_SIZE,SIZE_16,SIZE_16_PRICE
0,Brandega-48335,Days of Summer,Botann's,12345-94249-107,FFF-F-IC-P-64,77220-10116,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 16 oz,Frank's Famous Flavors Black Walnut 16 oz,F,...,-0.023,-0.10,0.06,-1,-0.037,2.05119,74.664,131.27616,0,0.0
1,Brandega-48335,Days of Summer,Botann's,12345-94249-107,FFF-F-IC-P-64,77220-10116,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 16 oz,Frank's Famous Flavors Black Walnut 16 oz,N,...,-0.023,0.01,0.06,1,0.073,2.23407,38.628,142.98048,0,0.0
2,Brandega-48335,Days of Summer,Botann's,12345-79914-108,FFF-F-IC-P-64,77220-10116,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 16 oz,Frank's Famous Flavors Black Walnut 16 oz,F,...,-0.023,-0.10,0.06,1,-0.014,2.15982,70.992,138.22848,0,0.0
3,Brandega-48335,Days of Summer,Botann's,12345-79914-108,FFF-F-IC-P-64,77220-10116,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 16 oz,Frank's Famous Flavors Black Walnut 16 oz,N,...,-0.023,0.01,0.06,-1,0.096,2.64136,32.544,169.04704,0,0.0
4,Brandega-48335,Days of Summer,Botann's,12345-90005-109,FFF-F-IC-P-64,77220-10116,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 16 oz,Frank's Famous Flavors Black Walnut 16 oz,F,...,-0.023,-0.10,0.06,1,-0.052,2.24076,68.256,143.40864,0,0.0


In [57]:
v=v.drop(['PRICE_R','UNITS_R'], axis=1)
v = pd.concat([v, pricev,qv], axis=1)
v['PRICE_POINT']='V'
stuff=df_total4[['Promo_ID','Retailer_Store_ID','Product_Unit_UPC','TPP_Type']]
v = pd.concat([v, stuff], axis=1)

In [58]:

w=w.drop(['PRICE_R','UNITS_R'], axis=1)
w = pd.concat([w, pricew,qw], axis=1)
w['PRICE_POINT']='W'

w = pd.concat([w, stuff], axis=1)

y=y.drop(['PRICE_R','UNITS_R'], axis=1)
y = pd.concat([y, pricey,qy], axis=1)
y['PRICE_POINT']='Y'
y = pd.concat([y, stuff], axis=1)

z=z.drop(['PRICE_R','UNITS_R'], axis=1)
z = pd.concat([z, pricez,qz], axis=1)
z['PRICE_POINT']='Z'
z = pd.concat([z, stuff], axis=1)

x['PRICE_POINT']='X'
x = pd.concat([x, stuff], axis=1)





/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [59]:
xxx=pd.concat([v,w,x,y,z],axis=0)

In [60]:
#build the ols model

independent=xxx[['PRICE_R','TOTAL_R','SIZE','PRICE_SIZE']]
P=xxx['PRICE_POINT']
stuff=xxx[['Promo_ID','Retailer_Store_ID','Product_Unit_UPC','TPP_Type']]

independent = sm.add_constant(independent, prepend=False)

dependent=xxx['UNITS_R']

In [61]:
mod = sm.OLS(dependent, independent)

In [62]:
res = mod.fit()

In [63]:
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                UNITS_R   R-squared:                       0.281
Model:                            OLS   Adj. R-squared:                  0.281
Method:                 Least Squares   F-statistic:                 4.606e+04
Date:                Thu, 17 Jan 2019   Prob (F-statistic):               0.00
Time:                        17:47:39   Log-Likelihood:            -2.5961e+06
No. Observations:              471510   AIC:                         5.192e+06
Df Residuals:                  471505   BIC:                         5.192e+06
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
PRICE_R      -11.2438      0.203    -55.431      0.0

In [64]:
ypred = pd.DataFrame(res.predict(independent))
ypred.columns = ['P_UNITS']

In [65]:
model_output=pd.concat([ypred,independent,stuff,P], axis=1)
model_output.head()

,P_UNITS,PRICE_R,TOTAL_R,SIZE,PRICE_SIZE,const,Promo_ID,Retailer_Store_ID,Product_Unit_UPC,TPP_Type,PRICE_POINT
0,119.601432,1.948630,-0.037,64.0,131.27616,1.0,Brandega-48335,12345-94249-107,77220-10116,F,V
1,67.561230,2.122367,0.073,64.0,142.98048,1.0,Brandega-48335,12345-94249-107,77220-10116,N,V
2,104.520683,2.051829,-0.014,64.0,138.22848,1.0,Brandega-48335,12345-79914-108,77220-10116,F,V
3,34.662162,2.509292,0.096,64.0,169.04704,1.0,Brandega-48335,12345-79914-108,77220-10116,N,V
4,113.900076,2.128722,-0.052,64.0,143.40864,1.0,Brandega-48335,12345-90005-109,77220-10116,F,V


In [66]:
df_main_stuff=df_total4.drop(['PRICE_R','TOTAL_R','SIZE','PRICE_SIZE'], axis=1)
df_main_stuff.columns

Index(['Promo_ID', 'Promo_Name', 'Retailer_Name', 'Retailer_Store_ID',
       'Product_Group_ID', 'Product_Unit_UPC', 'Product_Brand',
       'Product_Brand_Category_Desc', 'Product_Unit_Description', 'TPP_Type',
       'TPP_Start', 'TPP_End', 'Product_Units_Per_Case',
       'Product_CPG_COGS_Price', 'CASES', 'GROUP', 'UNITS', 'date', 'year',
       'month', 'Retailer_R', 'PRODUCT_GROUP_ID_R', 'Product_Unit_UPC_R',
       'Promo_Name_R', 'GROUP_R', 'month_R', 'randNumCol', 'UNITS_R',
       'SIZE_16', 'SIZE_16_PRICE'],
      dtype='object')

In [67]:
df_main_stuff=df_main_stuff.drop_duplicates(subset=['Promo_ID','Retailer_Store_ID','Product_Unit_UPC','TPP_Type'], keep='first', inplace=False)
df_main_stuff.shape

(94302, 30)

In [68]:
df_total5 =model_output.merge(df_main_stuff, on=['Promo_ID','Retailer_Store_ID','Product_Unit_UPC','TPP_Type'], how='inner')
df_total5.shape

(471510, 37)

In [69]:
df_total5.describe()

,P_UNITS,PRICE_R,TOTAL_R,SIZE,PRICE_SIZE,const,Product_Units_Per_Case,Product_CPG_COGS_Price,CASES,UNITS,...,Retailer_R,PRODUCT_GROUP_ID_R,Product_Unit_UPC_R,Promo_Name_R,GROUP_R,month_R,randNumCol,UNITS_R,SIZE_16,SIZE_16_PRICE
count,471510.000000,471510.000000,471510.000000,471510.000000,471510.000000,471510.0,471510.000000,471510.000000,471510.000000,471510.000000,...,471510.000000,471510.0000,471510.000000,471510.0000,471510.000,471510.000000,471510.000000,471510.000000,471510.000000,471510.000000
mean,86.642472,3.952321,0.021504,40.000509,125.195978,1.0,9.000064,3.214967,9.749740,78.748871,...,0.014861,-0.0025,0.021643,0.0150,-0.045,0.017500,-0.005578,78.765883,0.499989,3.246805
std,37.230570,1.973979,0.077481,24.000025,24.645955,0.0,3.000003,1.535002,8.863204,63.172768,...,0.025770,0.0095,0.022860,0.0301,0.055,0.031918,0.999986,63.520845,0.500001,3.256487
min,-27.704110,1.333920,-0.206000,16.000000,79.663680,1.0,6.000000,1.680000,3.000000,30.000000,...,-0.030000,-0.0120,-0.029000,-0.0230,-0.100,-0.030000,-1.000000,23.880000,0.000000,0.000000
25%,61.835967,2.050549,-0.032000,16.000000,101.666720,1.0,6.000000,1.680000,4.000000,36.000000,...,-0.007000,-0.0120,0.007000,-0.0230,-0.100,-0.030000,-1.000000,39.060000,0.000000,0.000000
50%,87.848786,2.846210,0.020000,64.000000,123.642240,1.0,12.000000,1.680000,6.000000,48.000000,...,0.017000,-0.0120,0.018000,-0.0050,-0.045,0.020000,-1.000000,47.280000,0.000000,0.000000
75%,111.086582,5.786901,0.083000,64.000000,144.831360,1.0,12.000000,4.750000,11.000000,72.000000,...,0.035000,0.0070,0.049000,0.0510,0.010,0.060000,1.000000,81.648000,1.000000,6.310000
max,208.500617,8.223320,0.204000,64.000000,182.311680,1.0,12.000000,4.750000,37.000000,228.000000,...,0.059000,0.0070,0.057000,0.0510,0.010,0.060000,1.000000,253.992000,1.000000,6.830000


In [70]:
df_total5.loc[df_total5['P_UNITS'] < 0, 'P_UNITS'] = 0
df_total5.describe()

,P_UNITS,PRICE_R,TOTAL_R,SIZE,PRICE_SIZE,const,Product_Units_Per_Case,Product_CPG_COGS_Price,CASES,UNITS,...,Retailer_R,PRODUCT_GROUP_ID_R,Product_Unit_UPC_R,Promo_Name_R,GROUP_R,month_R,randNumCol,UNITS_R,SIZE_16,SIZE_16_PRICE
count,471510.000000,471510.000000,471510.000000,471510.000000,471510.000000,471510.0,471510.000000,471510.000000,471510.000000,471510.000000,...,471510.000000,471510.0000,471510.000000,471510.0000,471510.000,471510.000000,471510.000000,471510.000000,471510.000000,471510.000000
mean,86.697795,3.952321,0.021504,40.000509,125.195978,1.0,9.000064,3.214967,9.749740,78.748871,...,0.014861,-0.0025,0.021643,0.0150,-0.045,0.017500,-0.005578,78.765883,0.499989,3.246805
std,37.092587,1.973979,0.077481,24.000025,24.645955,0.0,3.000003,1.535002,8.863204,63.172768,...,0.025770,0.0095,0.022860,0.0301,0.055,0.031918,0.999986,63.520845,0.500001,3.256487
min,0.000000,1.333920,-0.206000,16.000000,79.663680,1.0,6.000000,1.680000,3.000000,30.000000,...,-0.030000,-0.0120,-0.029000,-0.0230,-0.100,-0.030000,-1.000000,23.880000,0.000000,0.000000
25%,61.835967,2.050549,-0.032000,16.000000,101.666720,1.0,6.000000,1.680000,4.000000,36.000000,...,-0.007000,-0.0120,0.007000,-0.0230,-0.100,-0.030000,-1.000000,39.060000,0.000000,0.000000
50%,87.848786,2.846210,0.020000,64.000000,123.642240,1.0,12.000000,1.680000,6.000000,48.000000,...,0.017000,-0.0120,0.018000,-0.0050,-0.045,0.020000,-1.000000,47.280000,0.000000,0.000000
75%,111.086582,5.786901,0.083000,64.000000,144.831360,1.0,12.000000,4.750000,11.000000,72.000000,...,0.035000,0.0070,0.049000,0.0510,0.010,0.060000,1.000000,81.648000,1.000000,6.310000
max,208.500617,8.223320,0.204000,64.000000,182.311680,1.0,12.000000,4.750000,37.000000,228.000000,...,0.059000,0.0070,0.057000,0.0510,0.010,0.060000,1.000000,253.992000,1.000000,6.830000


In [71]:
df_total5=df_total5.rename(index=str, columns={"PRICE_R": "PRICE"})

#tips = df_total5.sort_values(['Promo_ID', 'Retailer_Store_ID', 'Product_Unit_UPC'])
df_total5.head()

,P_UNITS,PRICE,TOTAL_R,SIZE,PRICE_SIZE,const,Promo_ID,Retailer_Store_ID,Product_Unit_UPC,TPP_Type,...,Retailer_R,PRODUCT_GROUP_ID_R,Product_Unit_UPC_R,Promo_Name_R,GROUP_R,month_R,randNumCol,UNITS_R,SIZE_16,SIZE_16_PRICE
0,119.601432,1.948630,-0.037,64.0,131.27616,1.0,Brandega-48335,12345-94249-107,77220-10116,F,...,-0.012,-0.012,0.05,-0.023,-0.1,0.06,-1,74.664,0,0.0
1,121.907752,1.743511,-0.037,64.0,131.27616,1.0,Brandega-48335,12345-94249-107,77220-10116,F,...,-0.012,-0.012,0.05,-0.023,-0.1,0.06,-1,74.664,0,0.0
2,118.448272,2.051190,-0.037,64.0,131.27616,1.0,Brandega-48335,12345-94249-107,77220-10116,F,...,-0.012,-0.012,0.05,-0.023,-0.1,0.06,-1,74.664,0,0.0
3,120.754592,1.846071,-0.037,64.0,131.27616,1.0,Brandega-48335,12345-94249-107,77220-10116,F,...,-0.012,-0.012,0.05,-0.023,-0.1,0.06,-1,74.664,0,0.0
4,123.060912,1.640952,-0.037,64.0,131.27616,1.0,Brandega-48335,12345-94249-107,77220-10116,F,...,-0.012,-0.012,0.05,-0.023,-0.1,0.06,-1,74.664,0,0.0


In [72]:
#keep the relevant variables
df_total5 = df_total5[['Promo_ID',
 'Promo_Name',
 'PRICE_POINT',
'Retailer_Name',
 'Retailer_Store_ID',
 'Product_Group_ID',
 'Product_Unit_UPC',
 'Product_Brand',
 'Product_Brand_Category_Desc',
 'Product_Unit_Description',
 'PRICE',
 'TPP_Type',
 'TPP_Start',
 'TPP_End',
 'Product_Units_Per_Case',
 'Product_CPG_COGS_Price',
 'SIZE',
 'CASES',
 'GROUP',
 'UNITS',     
 'P_UNITS']]

In [73]:
#capitalize the field names to load in dash
df_total5=df_total5.rename(str.upper, axis='columns')

In [74]:
df_total5.PRICE_POINT.value_counts(dropna = False).sort_index()

V    94302
W    94302
X    94302
Y    94302
Z    94302
Name: PRICE_POINT, dtype: int64

In [75]:
#De-Dup the file
df_total5=df_total5.drop_duplicates(subset=['PROMO_ID','RETAILER_STORE_ID','PRODUCT_UNIT_UPC','GROUP','PRICE_POINT'], keep='first', inplace=False)
df_total5.shape
df_total5.head()


,PROMO_ID,PROMO_NAME,PRICE_POINT,RETAILER_NAME,RETAILER_STORE_ID,PRODUCT_GROUP_ID,PRODUCT_UNIT_UPC,PRODUCT_BRAND,PRODUCT_BRAND_CATEGORY_DESC,PRODUCT_UNIT_DESCRIPTION,...,TPP_TYPE,TPP_START,TPP_END,PRODUCT_UNITS_PER_CASE,PRODUCT_CPG_COGS_PRICE,SIZE,CASES,GROUP,UNITS,P_UNITS
0,Brandega-48335,Days of Summer,V,Botann's,12345-94249-107,FFF-F-IC-P-64,77220-10116,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 16 oz,Frank's Famous Flavors Black Walnut 16 oz,...,F,2019-07-07,2019-07-13,12.0,1.68,64.0,6.0,ONSALE,72.0,119.601432
1,Brandega-48335,Days of Summer,W,Botann's,12345-94249-107,FFF-F-IC-P-64,77220-10116,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 16 oz,Frank's Famous Flavors Black Walnut 16 oz,...,F,2019-07-07,2019-07-13,12.0,1.68,64.0,6.0,ONSALE,72.0,121.907752
2,Brandega-48335,Days of Summer,X,Botann's,12345-94249-107,FFF-F-IC-P-64,77220-10116,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 16 oz,Frank's Famous Flavors Black Walnut 16 oz,...,F,2019-07-07,2019-07-13,12.0,1.68,64.0,6.0,ONSALE,72.0,118.448272
3,Brandega-48335,Days of Summer,Y,Botann's,12345-94249-107,FFF-F-IC-P-64,77220-10116,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 16 oz,Frank's Famous Flavors Black Walnut 16 oz,...,F,2019-07-07,2019-07-13,12.0,1.68,64.0,6.0,ONSALE,72.0,120.754592
4,Brandega-48335,Days of Summer,Z,Botann's,12345-94249-107,FFF-F-IC-P-64,77220-10116,Frank's Famous Flavors,Frank's Famous Flavors Ice Cream Premium 16 oz,Frank's Famous Flavors Black Walnut 16 oz,...,F,2019-07-07,2019-07-13,12.0,1.68,64.0,6.0,ONSALE,72.0,123.060912


In [76]:
# The code was removed by Watson Studio for sharing.

In [77]:
df_load=df_total5

In [78]:
#load records to dashdb

import ibmdbpy
from ibmdbpy import IdaDataBase
idadb = idadb = IdaDataBase(dsn="DASHDB;Database=BLUDB;Hostname=" + credentials_1["host"] + ";PROTOCOL=TCPIP;UID=" + credentials_1["username"] + ";PWD=" + credentials_1["password"])

In [79]:
 idadb.as_idadataframe(df_load, "ICE_CREAM_FINAL_PRODUCTION",clear_existing = True)

DataFrame will be splitted into 1241 chunks. (380 rows per chunk)
Uploaded: 1241/1241... [DONE]


In [80]:
#spark_df = sqlContext.createDataFrame(df_total5)

In [81]:
#### this inserts records


#from ingest.Connectors import Connectors

#dashdbsaveoption = {
 #                    Connectors.DASHDB.HOST              : 'dashdb-entry-yp-dal09-08.services.dal.bluemix.net',
  #                   Connectors.DASHDB.DATABASE          : 'BLUDB',
   #                  Connectors.DASHDB.USERNAME          : 'dash10112',
    #                 Connectors.DASHDB.PASSWORD          : '-rMyl8IIE1~n',
     #                Connectors.DASHDB.TARGET_TABLE_NAME : 'ICE_CREAM',
      #               Connectors.DASHDB.TARGET_WRITE_MODE : 'merge' }

#spark_df = spark_df.write.format("com.ibm.spark.discover").options(**dashdbsaveoption).save()

In [82]:
#### this deletes then inserts records


#from ingest.Connectors import Connectors

#dashdbsaveoption = {
 #                    Connectors.DASHDB.HOST              : 'dashdb-entry-yp-dal09-08.services.dal.bluemix.net',
  #                   Connectors.DASHDB.DATABASE          : 'BLUDB',
   #                  Connectors.DASHDB.USERNAME          : 'dash10112',
    #                 Connectors.DASHDB.PASSWORD          : '-rMyl8IIE1~n',
     #                Connectors.DASHDB.TARGET_TABLE_NAME : 'DASH10112.ICE_CREAM',
      #               Connectors.DASHDB.TARGET_TABLE_ACTION : 'replace' }

#spark_df = spark_df.write.format("com.ibm.spark.discover").options(**dashdbsaveoption).save()

In [83]:
#This reads records


#from ingest.Connectors import Connectors
#from pyspark.sql import SQLContext
#sqlContext = SQLContext(sc)

#dashDBloadOptions = { Connectors.DASHDB.HOST              : 'dashdb-entry-yp-dal09-08.services.dal.bluemix.net',
 #                     Connectors.DASHDB.DATABASE          : 'BLUDB',
  #                    Connectors.DASHDB.USERNAME          : 'dash10112',
   #                   Connectors.DASHDB.PASSWORD          : '-rMyl8IIE1~n',
    #                  Connectors.DASHDB.SOURCE_TABLE_NAME : 'ICE_CREAM'}

#dashdbDF = sqlContext.read.format("com.ibm.spark.discover").options(**dashDBloadOptions).load()
#dashdbDF.printSchema()
#dashdbDF.show()


In [84]:
#abcd=dashdbDF.toPandas() 

In [85]:
#abcd.shape
#abcd.info()